In [ ]:
import os

In [ ]:
directory = "results/classifications/20211022/"
suffix= '20211022'

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       "-co NUM_THREADS=30 "\
       "--config GDAL_CACHEMAX 512 "
       + "Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.vrt " + "Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.tif")

## Clip mosaic to AEZ

In [ ]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

In [ ]:
create_local_dask_cluster()

In [ ]:
aez_region = 'data/Southern_SE.shp'
# aez_region = 'data/zambia.shp'
pred_tif = "results/classifications/"+suffix+"/Southeast_gm_mads_two_seasons_"+suffix+"_mosaic.tif"
chunks={'x':5000, 'y':5000}

In [ ]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [ ]:
aez = xr_rasterize(aez, da)

In [ ]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [ ]:
%%time
da = da.compute()

In [ ]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_clipped.tif',
          overwrite=True)

In [ ]:
# write_cog(aez.squeeze(),
#           'WDPA_southern.tif',
#           overwrite=True)

In [ ]:
# write_cog(da.squeeze(),
#           "Zambia_cropmask_"+suffix+".tif",
#           overwrite=True)